In [1]:
from xcube_gedi.store import GediDataStore

In [2]:
import xcube_gedi
xcube_gedi.__version__

'0.1.0.dev'

In [3]:
gedi = GediDataStore()

In [4]:
gedi.open_data()

<xarray.Dataset> Size: 3kB
Dimensions:         (shot_number: 5, profile_points: 101)
Coordinates:
  * shot_number     (shot_number) uint64 40B 234911100300153462 ... 234911100...
  * profile_points  (profile_points) int64 808B 0 1 2 3 4 5 ... 96 97 98 99 100
    latitude        (shot_number) float64 40B 50.72 50.72 50.72 50.72 50.72
    longitude       (shot_number) float64 40B 8.948 8.949 8.95 8.95 8.951
    time            (shot_number) datetime64[ns] 40B 2023-02-04 ... 2023-02-04
Data variables:
    agbd            (shot_number) float32 20B 53.25 71.06 70.14 48.32 144.3
    rh              (shot_number, profile_points) float32 2kB -3.85 ... 30.04